In [ ]:
from gurobipy import *

Given a set of commodities $h \in H$

In [ ]:
commodities = ['Pencils', 'Pens']

A set of nodes $j \in J$

In [ ]:
nodes = ['Detroit', 'Denver', 'Boston', 'New York', 'Seattle']

Arcs $(i, j) \in A$ with associated capacities $u_{ij}$

In [ ]:
arcs, capacity = multidict({
  ('Detroit', 'Boston'):   100,
  ('Detroit', 'New York'):  80,
  ('Detroit', 'Seattle'):  120,
  ('Denver',  'Boston'):   120,
  ('Denver',  'New York'): 120,
  ('Denver',  'Seattle'):  120 })
arcs = tuplelist(arcs)

Costs $c_{hij}$ to ship a unit of commodity $h$ on arc $(i, j)$

In [ ]:
cost = {
  ('Pencils', 'Detroit', 'Boston'):   10,
  ('Pencils', 'Detroit', 'New York'): 20,
  ('Pencils', 'Detroit', 'Seattle'):  60,
  ('Pencils', 'Denver',  'Boston'):   40,
  ('Pencils', 'Denver',  'New York'): 40,
  ('Pencils', 'Denver',  'Seattle'):  30,
  ('Pens',    'Detroit', 'Boston'):   20,
  ('Pens',    'Detroit', 'New York'): 20,
  ('Pens',    'Detroit', 'Seattle'):  80,
  ('Pens',    'Denver',  'Boston'):   60,
  ('Pens',    'Denver',  'New York'): 70,
  ('Pens',    'Denver',  'Seattle'):  30 }

Net demand (supply if negative) for commodity $h$ at node $j$, $d_{hj}$

In [ ]:
demand = {
  ('Pencils', 'Detroit'):   -50,
  ('Pencils', 'Denver'):    -60,
  ('Pencils', 'Boston'):   50,
  ('Pencils', 'New York'): 50,
  ('Pencils', 'Seattle'):  10,
  ('Pens',    'Detroit'):   -60,
  ('Pens',    'Denver'):    -40,
  ('Pens',    'Boston'):   40,
  ('Pens',    'New York'): 30,
  ('Pens',    'Seattle'):  30 }

In [ ]:
m = Model('netflow')

Decision variable $x_{hij} =$ number of units of commodity $h$ to ship on arc $(i, j)$

In [ ]:
flow = {}
for h in commodities:
    for i,j in arcs:
        flow[h,i,j] = m.addVar(ub=capacity[i,j], obj=cost[h,i,j],
                               name='flow.{0}.{1}.{2}'.format(h, i, j))
m.update()

Arc capacity constraints $\sum_{(i, j) \in A} x_{hij} \le u_{ij},\;(i, j) \in A$

In [ ]:
for i,j in arcs:
    m.addConstr(quicksum(flow[h,i,j] for h in commodities) <= capacity[i,j],
                'cap.{0}.{1}'.format(i, j))

Flow balance constraints $\sum_{i \in RS(j)} x_{hij} - \sum_{k \in FS(j)} x_{hjk} = d_{hj},\;\;h \in H,\;j \in J$

In [ ]:
# Flow conservation constraints
for h in commodities:
    for j in nodes:
        m.addConstr(
          quicksum(flow[h,i,j] for i,j in arcs.select('*',j)) -
          quicksum(flow[h,j,k] for j,k in arcs.select(j,'*')) ==
          demand[h, j], 'node.{0}.{1}'.format(h, j))

In [ ]:
m.optimize()

if m.status == GRB.status.OPTIMAL:
    solution = m.getAttr('x', flow)
    for h in commodities:
        print('\nOptimal flows for %s:' % h)
        for i,j in arcs:
            if solution[h,i,j] > 0:
                print('{0} -> {1}: {2}'.format(i, j, solution[h,i,j]))


In [ ]:
!cat network_flow.lp